# Archivo Historico de Localidades
(Historical Archive of Localities)

El Archivo Historico de Localidades es una base de datos de INEGI con mas de 2,138,702 registros de los diferentes censo y conteos que se han realizado en la Republica Mexicana desde el año 1900 a la fecha.

In [1]:
# Librerias

from zipfile import ZipFile
from io import BytesIO
import pandas as pd
import numpy as np
import os

In [2]:
verbose = True  # Imprimir notas de ejecución

## Funciones

In [3]:
def extraer(ruta_zip, sub_zip, extraer, verbose = True):
    '''
    Función que extrae una archivo de los zips estatales y 
    lo regresa en formato de dataframe de pandas.
    '''
    archivo_extraer = sub_zip[0:5] + extraer + '.csv' # Nombre del archivo a extraer
    #print(archivo_extraer)
    
    with ZipFile(ruta_zip, 'r') as zip_file:         # Zip general   
        sub_zip = BytesIO(zip_file.read(sub_zip))    # Leer archivo de zip estatal
        
        with ZipFile(sub_zip, 'r') as info_estatal:  # Zip estatal
            lista_estatal = info_estatal.namelist()  # Archivos contenidos

            with info_estatal.open(archivo_extraer, 'r') as archivo:   # Archivo de interes
                habitantes = pd.read_csv(archivo, encoding = 'cp1252', # Tiene esta codificación
                                         dtype={                       # Definir como se importan las columnas
                                             'CVE_GEOEST': str         # CVE_GEOEST como string
                                         })
            if verbose:
#                print('Contenido del zip estatal / State zip content:', end = '\n\n')
#                print(lista_estatal, end = '\n\n')
                print(f'Extrayendo archivo: {archivo_extraer}...')
    
    return habitantes


In [4]:
dir_datos = os.environ.get('DATA_DIR')                   # Directorio de los datos
sub_dir = 'AHL'                                          # Sub-directorio del zip
nom_archivo = 'AHL.zip'                                  # Nombre del zip
ruta_zip = os.path.join(dir_datos, sub_dir, nom_archivo) # Ruta del archivo zip


Contenido del archivo principal - Content of main archive:

In [5]:
with ZipFile(ruta_zip, 'r') as zip_file:
    sub_zips = zip_file.namelist()
    if verbose:
        print('Contenido del zip:', end = '\n\n')
        print(sub_zips)

Contenido del zip:

['AHL01.zip', 'AHL02.zip', 'AHL03.zip', 'AHL04.zip', 'AHL05.zip', 'AHL06.zip', 'AHL07.zip', 'AHL08.zip', 'AHL09.zip', 'AHL10.zip', 'AHL11.zip', 'AHL12.zip', 'AHL13.zip', 'AHL14.zip', 'AHL15.zip', 'AHL16.zip', 'AHL17.zip', 'AHL18.zip', 'AHL19.zip', 'AHL20.zip', 'AHL21.zip', 'AHL22.zip', 'AHL23.zip', 'AHL24.zip', 'AHL25.zip', 'AHL26.zip', 'AHL27.zip', 'AHL28.zip', 'AHL29.zip', 'AHL30.zip', 'AHL31.zip', 'AHL32.zip']


## Archivo habitantes

En general INEGI utiliza la codificacion 'utf-8', pero hay veces que los datos se exportan en la codificación que el sistema operativo Windows usa para el español llamado 'Windows-1252' (tambien con codigo 'cp1252'). Esto sucede generalmente cuando manipula y exporta los datos desde Excel (o tal vez Access).  

--------------------------------------------

INEGI normally encodes its data in 'utf-8', but sometimes it is in 'Windows-1252' (which is the same as 'cp1252') when its manipulated and exported from Excel (or maybe Access). 

In [6]:
habitantes = pd.concat(
    [extraer(ruta_zip, sub_zip, 'Habitantes', False) for sub_zip in sub_zips])


In [12]:
# Información general

#habitantes.memory_usage().sum() / 1_000_000_000  # Memoria utilizada por el archivo
habitantes.info(show_counts = True)              # Valores nulos y tipo de datos por columna
#habitantes.describe()                            # Estadisticas por columna





<class 'pandas.core.frame.DataFrame'>
Int64Index: 2138702 entries, 0 to 55293
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   CLAVE       2138702 non-null  int64 
 1   CVE_GEOEST  2138702 non-null  object
 2   EVE_CENSAL  2138702 non-null  int64 
 3   INDICE_HAB  2138702 non-null  int64 
 4   FUENTE      2138702 non-null  object
 5   TOT_HAB     2138702 non-null  object
 6   TOT_HOM     2138702 non-null  object
 7   TOT_MUJ     2138702 non-null  object
dtypes: int64(3), object(5)
memory usage: 146.9+ MB


In [14]:
habitantes.head()

,CLAVE,CVE_GEOEST,EVE_CENSAL,INDICE_HAB,FUENTE,TOT_HAB,TOT_HOM,TOT_MUJ
0,1,010010001,1940,14962331,Censo,"82,234","37,821","44,413"
1,1,010010001,1930,14962332,Censo,"62,244","28,687","33,557"
2,1,010010001,1900,14962333,Censo,"35,052","16,229","18,823"
3,1,010010001,1970,14962334,Censo,"181,277",-,-
4,1,010010001,2005,14962335,Conteo,"663,671","319,649","344,022"


Las claves en INEGI deben de ser manejadas como texto. Esto es cierto para CVE_GEOEST, que representa la clave geoestadistica de una localidad especifica. Si se importa como numero entonces a clave '010010001' se importará como 10010001. En otras palabras se eliminará el cero al principio de la clave que es necesario porque representa al estado en el que se encuentra la localidad y hay 32 en el pais.

Cada localidad tiene una clave que esta formada por 9 caracteres numericos. Los primeros dos representan la clave del estado ('01'), los siguientes tres caracteres representan al municipio ('001'), y los ultimos cuatro numeros representan la clave de la localidad ('0001'). Los Estados estan organizados por orden alfabetico. Los municipios y localidades no son consistentes con este criterio porque cuando se crean nuevos registros la numeración no es reiniciada. Solo hay una regla de nomenclatura para las localidades, la clave '0001' siempre representará la cabecera municipal.


--------------------------------------------

Keys in INEGI should be treated like strings. This is true ofre CVE_GEOEST that representas the geo-statistical key of a specific locality. If this field is imported like a number then the key '010010001' will be imported with the format 10010001. It will eliminate the zero at hte begining an it is necesario because it represents the state in with de locality is in and there are 32 in the country.

Each locality has a key that is formed by 9 numerical caracters. The first two represent the state key ('01'), the next three represent the municipality ('001'), and the las four numberes represent the key of the locality ('0001'). The States are organized in alphabetical order. The municipalites and localities are not consistent with this criteria because when a new entitty is created the numeration is no reiniciated.There is only one nomenclature rull in localities, the key '0001' will always represent the municipal head (the locality where the municipal government resides).

In [33]:
#len(habitantes.CVE_GEOEST.unique())
#habitantes.CVE_GEOEST.value_counts()
GEOEST_CVE = habitantes['CVE_GEOEST'] + habitantes['CLAVE'].apply(str)
print(len(habitantes.CVE_GEOEST.unique()), len(GEOEST_CVE.unique()))

408754 408754


La columna CLAVE es un numero consecutivo que es diferente para cada localidad del estado. Por eso tiene la misma cantidad de valores unicos que CVE_GEOEST.

EVE_CENSAL representa el evento censal donde se obuvieron las mediciones de las columnas TOT_HAB (Total de habitantes), TOT_HOM (Total de hombres), y TOT_MUJ (Total de mujeres).

Los valores unicos de FUENTE representan el tipo de evento censal.

--------------------------------

CLAVE is a consecutive number that is diferent for each locality of the state. That is why it has the same cuantity of values as CVE_GEOEST.

EVE_CENSAL represents the censal event where the data of TOT_HAB (Total inhabitants), TOT_HOM (Men total), and TOT_MUJ (Women total).

The unique values of FUENTE represent the tipe of censal event (Count or full Cense).

In [13]:
#sorted(habitantes.EVE_CENSAL.unique().tolist()) # Lista de valores unicos en EVE_CENSAL
habitantes.EVE_CENSAL.value_counts()
#habitantes[habitantes.EVE_CENSAL==2]

#habitantes.CLAVE.value_counts().unique()


2010    279187
2020    275867
2005    265585
2000    255718
1995    252784
1990    173771
1980    129895
1970     98154
1960     87878
1950     75029
1940     69733
1930     57411
1921     43851
1910     41045
1900     32605
1939       179
2            8
1920         1
2013         1
Name: EVE_CENSAL, dtype: int64

In [16]:
lista_columnas = habitantes.columns.values.tolist()

for col in lista_columnas[ :5]: 
    print(f'Cantidad de valores unicos de {col}: {len(habitantes[col].unique())}')
    
print()
print('Valores unicos de EVE_CENSAL:', end = '\n\n')
print(habitantes['EVE_CENSAL'].unique(), end = '\n\n')

print('Valores unicos de FUENTE:', end = '\n\n')
print(habitantes['FUENTE'].unique(), end = '\n\n')

print('Valores unicos de INDICE_HAB:', end = '\n\n')
print(habitantes['INDICE_HAB'].unique(), end = '\n\n')



Cantidad de valores unicos de CLAVE: 408754
Cantidad de valores unicos de CVE_GEOEST: 408754
Cantidad de valores unicos de EVE_CENSAL: 19
Cantidad de valores unicos de INDICE_HAB: 2138702
Cantidad de valores unicos de FUENTE: 2

Valores unicos de EVE_CENSAL:

[1940 1930 1900 1970 2005 1995 1950 2020 1960 2010 1910 2000 1980 1990
 1921 1939 1920    2 2013]

Valores unicos de FUENTE:

['Censo' 'Conteo']

Valores unicos de INDICE_HAB:

[14962331 14962332 14962333 ... 17101030 17101031 17101032]



In [ ]:
habitantes['TOT_HAB'] =pd.to_numeric(habitantes['TOT_HAB'], 
                                     errors='coerce')

habitantes['TOT_HOM'] =pd.to_numeric(habitantes['TOT_HOM'], 
                                     errors='coerce',
                                     downcast='integer')

habitantes['TOT_MUJ'] =pd.to_numeric(habitantes['TOT_MUJ'], 
                                     errors='coerce',
                                     downcast='integer')

In [ ]:
habitantes.loc[habitantes['EVE_CENSAL'] == 1921]